In [9]:
from __future__ import print_function
from IPython.utils.path import get_ipython_dir
print(get_ipython().profile_dir.location)

import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import talib
import itable
import ffn

from fintools.Parameters import Parameters
from fintools.set_start_end import set_start_end
from fintools.get_yahoo_prices import get_yahoo_prices
from fintools.compute_weights_RS_DM import compute_weights_RS_DM
from fintools.compute_weights_PMA import compute_weights_PMA
from fintools.endpoints import endpoints
from fintools.backtest import backtest
from fintools.monthly_return_table import monthly_return_table
from fintools.show_return_table import show_return_table

%matplotlib inline


###############################################################################
# useful utility routine
def side_by_side(*objs, **kwds):
    from pandas.core.common import adjoin
    space = kwds.get('space', 4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print (adjoin(space, *reprs))

/home/scubamut/.ipython/profile_default


In [11]:
# import pandas as pd
# from .Parameters import Parameters
# from .get_yahoo_prices import get_yahoo_prices
# from .endpoints import endpoints
# from .backtest import backtest

def compute_weights_RS_DM(name, parameters):

    print('Strategy : {}'.format(name))

    p = Parameters(parameters)

    prices = get_yahoo_prices(p)

    end_points = endpoints(period=p.frequency, trading_days=prices.index)
    prices_m = prices.loc[end_points]

    returns = prices_m[p.symbols].pct_change(p.rs_lookback)[p.rs_lookback:]

    if isinstance(p.risk_free, int):
        excess_returns = returns
    else:
        risk_free_returns = prices_m[p.risk_free].pct_change(p.rs_lookback)[p.rs_lookback:]
        excess_returns = returns.subtract(risk_free_returns, axis=0).dropna()

    absolute_momentum = prices_m[p.symbols].pct_change(p.risk_lookback)[p.risk_lookback:]
    absolute_momentum_rule = absolute_momentum > 0
    rebalance_dates = excess_returns.index.join(absolute_momentum_rule.index, how='inner')

    # relative strength ranking
    ranked = excess_returns.loc[rebalance_dates][p.symbols].rank(ascending=False, axis=1, method='dense')
    # elligibility rule - top n_top ranked securities
    elligible = ranked[ranked <= p.n_top] > 0

    # equal weight allocations
    elligible = elligible.multiply(1. / elligible.sum(1), axis=0)

    # downside protection
    weights = pd.DataFrame(0., index=elligible.index, columns=prices.columns)
    if p.cash_proxy == 'CASHX':
        weights[p.cash_proxy] = 0
        prices[p.cash_proxy] = 1.
    weights[p.symbols] = (elligible * absolute_momentum_rule).dropna()
    weights[p.cash_proxy] += 1 - weights[p.symbols].sum(axis=1)

    # backtest

    p_value, p_holdings, p_weights = backtest(prices, weights, 10000., offset=0, commission=10.)

    # p_value.plot(figsize=(15, 10), grid=True, legend=True, label=name)

    return p_value, p_holdings, p_weights, prices


In [12]:
def compute_weights_PMA(name, parameters):

    print(name)

    p = Parameters(parameters)

    prices = get_yahoo_prices(p)

    end_points = endpoints(period=p.frequency, trading_days=prices.index)
    prices_m = prices.loc[end_points]

    # elligibility rule
    SMA = prices_m.rolling(p.risk_lookback).mean().dropna()
    rebalance_dates = SMA.index
    rule = prices_m.loc[rebalance_dates][p.symbols] > SMA[p.symbols]

    # fixed weight allocation
    weights = p.allocations * rule

    # downside protection
    weights[p.cash_proxy] = 1 - weights[p.symbols].sum(axis=1)

    # backtest
    p_value, p_holdings, p_weights = backtest(prices, weights, 10000., offset=0, commission=10.)

    # p_value.plot(figsize=(15, 10), grid=True, legend=True, label=name)

    return p_value, p_holdings, p_weights, prices

In [13]:
# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until today.

start_date = '1986-01-01'
# start_date = '2000-01-01'
# end_date = '2016-12-31'
end_date = datetime.today().strftime('%Y-%m-%d')

strategies = {
    'RS0001': { 'symbols': ['CWB','HYG','MBB','IEF','HYD'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'rs_lookback': 1, 'risk_lookback': 1, 'n_top': 2, 'frequency': 'M',
              'cash_proxy': 'CASHX', 'risk_free': 0},
    'RS0002': {'symbols': ['HYD','QLTB','MBB'], 
               'prices':data_source, 'start':start_date, 'end':end_date,               
               'rs_lookback': 3, 'risk_lookback': 2, 'n_top': 1, 'frequency': 'M',
              'cash_proxy': 'CASHX', 'risk_free': 0},
    'RS0003': {'symbols': ['HYD','MBB','SHY'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'rs_lookback': 1, 'risk_lookback': 1, 'n_top': 1, 'frequency': 'Q',
              'cash_proxy': 'CASHX', 'risk_free': 0},    
    'DM0001': {'symbols': ['CWB','VYM','HYG','VHT','TLT','MBB','HYD','IEF'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'rs_lookback': 1, 'risk_lookback': 1, 'n_top': 3, 'frequency': 'M',
              'cash_proxy': 'CASHX', 'risk_free': 'SHY'},
    'DM0002': {'symbols': ['CWB','TLT','HYG','VHT','VNQ'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'rs_lookback': 1, 'risk_lookback': 1, 'n_top': 5, 'frequency': 'M',
              'cash_proxy': 'MBB', 'risk_free': 'SHY'},
    'PMA001': {'symbols': ['CWB','MBB'], 
               'prices':data_source, 'start':start_date, 'end':end_date,  
               'risk_lookback': 3, 'frequency': 'M', 'allocations': [0.6, 0.4],
              'cash_proxy': 'TLT'},
    'PMA002': {'symbols': ['CWB','VYM','HYG'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'risk_lookback': 3, 'frequency': 'M', 'allocations': [0.6, 0.2, 0.2],
              'cash_proxy': 'TLT'},
    'PMA003': {'symbols': ['VCVSX', 'FAGIX', 'VGHCX'], 
               'prices':data_source, 'start':start_date, 'end':end_date,
               'risk_lookback': 2, 'frequency': 'M', 'allocations': [1./3., 1./3., 1./3.],
              'cash_proxy': 'VUSTX'}
    }

In [15]:
name = 'RS0002'


In [16]:
compute_weights_RS_DM (name, strategies[name])

Strategy : RS0002


ValueError: Index contains duplicate entries, cannot reshape

In [18]:
p = Parameters(strategies[name])
p

In [19]:
prices = get_yahoo_prices(p)

ValueError: Index contains duplicate entries, cannot reshape

In [26]:
tickers

['HYD', 'QLTB', 'MBB']

In [47]:
d1 = data.DataReader(['HYD'], "yahoo", start, end)['Adj Close']
d1

Symbols,HYD
Date,
2009-02-05,15.677656
2009-02-06,15.743864
2009-02-09,15.767930
2009-02-10,15.767930
2009-02-11,15.749880
2009-02-12,15.779973
2009-02-13,15.785989
2009-02-17,15.779973
2009-02-18,15.816076


In [55]:
data.DataReader(['QLTB'], "yahoo", start, end)

Attributes,High,Low,Open,Close,Volume,Adj Close
Symbols,QLTB,QLTB,QLTB,QLTB,QLTB,QLTB
Date,,,,,,
2012-04-26,50.470001,50.209999,50.470001,50.209999,200.0,42.942619
2012-04-27,50.209999,50.209999,50.209999,50.209999,0.0,42.942619
2012-04-30,50.540001,50.500000,50.540001,50.500000,2200.0,43.190647
2012-05-01,50.500000,50.500000,50.500000,50.500000,0.0,43.190647
2012-05-02,50.650002,50.590000,50.639999,50.650002,1900.0,43.318924
2012-05-03,51.169998,50.720001,51.169998,50.730000,700.0,43.387363
2012-05-04,50.740002,50.740002,50.740002,50.740002,900.0,43.395912
2012-05-07,50.790001,50.759998,50.759998,50.790001,700.0,43.438671


In [48]:
d2 = data.DataReader(['QLTB'], "yahoo", start, end)['Adj Close']
d2

Symbols,QLTB
Date,
2012-04-26,42.942619
2012-04-27,42.942619
2012-04-30,43.190647
2012-05-01,43.190647
2012-05-02,43.318924
2012-05-03,43.387363
2012-05-04,43.395912
2012-05-07,43.438671
2012-05-08,43.489986


In [49]:
d3 = data.DataReader(['MBB'], "yahoo", start, end)['Adj Close']
d3

Symbols,MBB
Date,
2007-03-16,71.163094
2007-03-19,71.163094
2007-03-20,71.247757
2007-03-21,71.339478
2007-03-22,71.240707
2007-03-23,71.247757
2007-03-26,71.254837
2007-03-27,71.219528
2007-03-28,71.212494


In [61]:
pd.concat([d1, d2], axis=1)

ValueError: Shape of passed values is (2, 2399), indices imply (2, 2397)

In [58]:
pd.concat([d1,d2,d3])

,HYD,MBB,QLTB
Date,,,
2009-02-05,15.677656,NaN,NaN
2009-02-06,15.743864,NaN,NaN
2009-02-09,15.767930,NaN,NaN
2009-02-10,15.767930,NaN,NaN
2009-02-11,15.749880,NaN,NaN
2009-02-12,15.779973,NaN,NaN
2009-02-13,15.785989,NaN,NaN
2009-02-17,15.779973,NaN,NaN
2009-02-18,15.816076,NaN,NaN


In [32]:
start,end

('1986-01-01', '2018-08-13')

In [25]:
from pandas_datareader import data

if isinstance(p.prices, str):
    if p.prices == 'yahoo':
        tickers = p.symbols.copy()
        if p.cash_proxy != 'CASHX':
            tickers = list(set(tickers + [p.cash_proxy]))
        try:
            if isinstance(p.risk_free, str):
                tickers = list(set(tickers + [p.risk_free]))

        except:
            pass

        if p.start >= p.end:
            raise ('start must be < end')

        start, end = set_start_end()

        data_panel = data.DataReader(tickers, "yahoo", start, end)

        close = data_panel['Adj Close'].sort_index(ascending=True)

#          close.copy().dropna()
# else:
#      return p.prices

ValueError: Index contains duplicate entries, cannot reshape